In [22]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, concat_ws, collect_list

import pandas as pd
import numpy as np

In [2]:
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/23 10:53:14 INFO SparkEnv: Registering MapOutputTracker
24/04/23 10:53:14 INFO SparkEnv: Registering BlockManagerMaster
24/04/23 10:53:14 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
24/04/23 10:53:14 INFO SparkEnv: Registering OutputCommitCoordinator


In [3]:
# targets = "gs://open-targets-data-releases/24.03/output/etl/parquet/targets"
# targets = spark.read.parquet(targets)

## Target prioritisation validation based on Open Targets score for IBD

### Export Target-IBD associations from Open Targets platform

For this MVP I just parsed all IBD-related evidence from Open Targets platform interface (this step can be improved in future).
Sources of evidence: Genetic evidence, Animal models, Expression Atlas.

In [13]:
evidence_ibd_path = "OT-EFO_0003767-associated-targets-23_04_2024-v24_03.tsv"
evidence_ibd = pd.read_csv(evidence_ibd_path, delimiter='\t')

In [14]:
evidence_ibd

,symbol,globalScore,otGeneticsPortal,geneBurden,eva,genomicsEngland,gene2Phenotype,uniprotLiterature,uniprotVariants,orphanet,clingen,expressionAtlas,impc
0,NOD2,0.871262,0.8588080964977179,0.796881794134767,0.8947318257571876,0.9176140716135052,No data,0.7599134970145264,0.9922618426290432,No data,No data,0.1384615637209791,0.44197286682445364
1,IL10RA,0.817033,No data,No data,0.9191986192098088,0.9360722249447112,No data,0.607930797611621,0.9190679877429972,0.607930797611621,No data,0.06033086779867074,0.8753026343414111
2,IL10RB,0.787469,No data,No data,0.9239580284256436,0.9360722249447112,No data,0.7599134970145264,No data,0.607930797611621,No data,No data,0.33813110963158366
3,ADAM17,0.712603,No data,No data,0.8867538587177661,0.8274613634158176,No data,No data,No data,0.607930797611621,No data,No data,0.4209359281821015
4,ITGA4,0.705869,0.6793122187858052,No data,No data,No data,No data,No data,No data,No data,No data,0.008402928573624254,No data
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10845,SPACA9,0.001028,No data,No data,No data,No data,No data,No data,No data,No data,No data,0.008456242821253483,No data
10846,SRD5A3,0.001027,No data,No data,No data,No data,No data,No data,No data,No data,No data,0.008448729384493684,No data
10847,SLC22A15,0.001021,No data,No data,No data,No data,No data,No data,No data,No data,No data,0.008401000156881841,No data
10848,HS3ST1,0.001020,No data,No data,No data,No data,No data,No data,No data,No data,No data,0.008385945404384204,No data


Now we should exclude evidence sources that can be built on info about known IBD drug targets to make validation more sensible.

For **globalScore** calculation we will use:
- otGeneticsPortal
- geneBurden
- eva
- uniprotLiterature (?)
- uniprotVariants (?)
- impc
- expressionAtlas

*? - means we are not sure but decided to use it for the mvp*

In [30]:
# Lets calculate new globalScore

evidence_ibd_filter = evidence_ibd[['symbol',
                                    'otGeneticsPortal',
                                    'geneBurden',
                                    'eva',
                                    'uniprotLiterature',
                                    'uniprotVariants',
                                    'impc',
                                    'expressionAtlas']].set_index('symbol')


In [42]:
# Weight factors for data sources

weights = {
    'impc': 0.2,
    'expressionAtlas': 0.2,
    'europepmc': 0.2,
    'progeny': 0.5,
    'slapenrich': 0.5,
    'cancerBiomarkers': 0.5,
    'sysbio': 0.5,
}

In [60]:
def harmonic_sum_weighted(row):
    row = row.dropna()  # Dropping NaNs
    # Considering the weights in the score calculation
    scores = [(idx, score) for idx, score in row.items() if idx in weights and score>0]
    scores.sort(key=lambda x: -x[1]) # sorting by score (higher first without weights)
    # Calculating the harmonic sum and the max. theoretical harmonic sum
    harmonic_sum_score = sum(weights[x[0]]*x[1]/((idx+1)**2) for idx, x in enumerate(scores))
    max_theoretical_sum = sum(weights[x[0]]/((idx+1)**2) for idx, x in enumerate(scores))
    # Returning the normalised harmonic sum score
    return harmonic_sum_score / max_theoretical_sum if max_theoretical_sum != 0 else 0

In [61]:
evidence_ibd_new_score = evidence_ibd_filter.apply(pd.to_numeric, errors='coerce')  # Convert all columns to numeric, turn 'no data' into NaN
evidence_ibd_new_score['globalScoreNew'] = evidence_ibd_new_score.apply(harmonic_sum, axis=1)
evidence_ibd_new_score

,otGeneticsPortal,geneBurden,eva,uniprotLiterature,uniprotVariants,impc,expressionAtlas,globalScoreNew
symbol,,,,,,,,
NOD2,0.858808,0.796882,0.894732,0.759913,0.992262,0.441973,0.138462,0.930464
IL10RA,NaN,NaN,0.919199,0.607931,0.919068,0.875303,0.060331,0.879079
IL10RB,NaN,NaN,0.923958,0.759913,NaN,0.338131,NaN,0.846005
ADAM17,NaN,NaN,0.886754,NaN,NaN,0.420936,NaN,0.793590
ITGA4,0.679312,NaN,NaN,NaN,NaN,NaN,0.008403,0.545130
...,...,...,...,...,...,...,...,...
SPACA9,NaN,NaN,NaN,NaN,NaN,NaN,0.008456,0.008456
SRD5A3,NaN,NaN,NaN,NaN,NaN,NaN,0.008449,0.008449
SLC22A15,NaN,NaN,NaN,NaN,NaN,NaN,0.008401,0.008401


#### Checking if the formula gives the same result as in Platform

In [49]:
evidence_ibd_all_path = "OT-EFO_0003767-associated-targets-23_04_2024-v24_03_all.tsv"
evidence_ibd_all = pd.read_csv(evidence_ibd_all_path, delimiter='\t')

In [62]:
evidence_ibd_all_score = evidence_ibd_all.apply(pd.to_numeric, errors='coerce')  # Convert all columns to numeric, turn 'no data' into NaN
evidence_ibd_all_score['globalScoreNew'] = evidence_ibd_all_score.apply(harmonic_sum, axis=1)
evidence_ibd_all_score

,symbol,globalScore,otGeneticsPortal,geneBurden,eva,genomicsEngland,gene2Phenotype,uniprotLiterature,uniprotVariants,orphanet,...,crisprScreen,crispr,slapenrich,progeny,reactome,sysbio,europepmc,expressionAtlas,impc,globalScoreNew
0,NaN,0.871262,0.858808,0.796882,0.894732,0.917614,NaN,0.759913,0.992262,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.905744,0.138462,0.441973,0.949760
1,NaN,0.817033,NaN,NaN,0.919199,0.936072,NaN,0.607931,0.919068,0.607931,...,NaN,NaN,NaN,NaN,NaN,0.561181,0.684237,0.060331,0.875303,0.905827
2,NaN,0.787469,NaN,NaN,0.923958,0.936072,NaN,0.759913,NaN,0.607931,...,NaN,NaN,NaN,NaN,NaN,NaN,0.494565,NaN,0.338131,0.890998
3,NaN,0.712603,NaN,NaN,0.886754,0.827461,NaN,NaN,NaN,0.607931,...,NaN,NaN,NaN,NaN,NaN,NaN,0.611428,NaN,0.420936,0.836148
4,NaN,0.705869,0.679312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.264410,0.008403,NaN,0.856853
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10845,NaN,0.001028,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.008456,NaN,0.006971
10846,NaN,0.001027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.008449,NaN,0.006964
10847,NaN,0.001021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.008401,NaN,0.006925
10848,NaN,0.001020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.008386,NaN,0.006913


In [8]:
# All Target-Disease evidence from Open Targets Platform
evidence_path = "gs://open-targets-data-releases/24.03/output/etl/parquet/evidence"
evidence = spark.read.parquet(evidence_path)

# Filter to only IBD evidence
evidence_ibd = evidence.filter(
    (col("diseaseId") == "EFO_0003767") 
).persist()

24/04/23 11:03:10 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [9]:
evidence_ibd.show()

+------------+---------------+-------------+-------------------+--------+----------+----+---------------------------+---------------------------+---------------------------------+--------------------------------+-----------------+-------------+----------+--------------------+--------+-------------+---------------------+--------------+-----------------+--------+----------------+---------------+----------+--------+-------------------+----------+----------------+-----------------+-------------------+-------------------------+-------------------------------------+-------------------------------------+--------------+------+------------+-----------------+-----------+----------------------------+-------------------+--------------+---------+--------------------------------+--------------------------------+--------------+--------------+--------+-------------+---------+----------------------+---------------+----------+------------+-----------+-------------+----+------------------------+---------

In [4]:
# Target-Disease indirect (propagated) evidence from Open Targets Platform
evidence_path_ByDatatype = "gs://open-targets-data-releases/23.12/output/etl/parquet/associationByDatatypeIndirect"
evidence_ByDatatype = spark.read.parquet(evidence_path_ByDatatype)

# Filter to only IBD associations
evidence_ibd_ByDatatype = evidence_ByDatatype.filter(
    (col("diseaseId") == "EFO_0003767") 
).persist()

# targets_ibd_ByDatatype = evidence_ibd_ByDatatype \
#     .groupBy("targetId") \
#     .agg(concat_ws(";", collect_list("datatypeId")).alias("datatypeId")) 

In [7]:
evidence_ibd_ByDatatype.show()

+-----------+---------------+-------------------+--------------------+-------------+
|  diseaseId|       targetId|         datatypeId|               score|evidenceCount|
+-----------+---------------+-------------------+--------------------+-------------+
|EFO_0003767|ENSG00000000938|   affected_pathway| 0.47965739931556906|            2|
|EFO_0003767|ENSG00000000938|       animal_model|   0.262139759930131|            1|
|EFO_0003767|ENSG00000000938|     rna_expression| 0.03380037948545017|            1|
|EFO_0003767|ENSG00000000971|         literature| 0.02262853524443256|            3|
|EFO_0003767|ENSG00000001084|         literature|  0.0787843505875546|            7|
|EFO_0003767|ENSG00000001626|       animal_model|  0.5664840700383574|           32|
|EFO_0003767|ENSG00000001626|genetic_association| 0.31300093795421535|            1|
|EFO_0003767|ENSG00000001626|         literature|  0.6704393153532628|           47|
|EFO_0003767|ENSG00000001630|         literature|0.03039653988058

In [17]:
# Filter to only genetic_association (any), animal_model, rna_expression

targets_ibd_filter = targets_ibd_ByDatatype.filter(
        F.col("datatypeId").contains("genetic_association") |
        F.col("datatypeId").contains("animal_model") |
        F.col("datatypeId").contains("rna_expression"))

targets_ibd_pd = targets_ibd_filter.toPandas()
targets_ibd_pd

AnalysisException: Column 'datatypeId' does not exist. Did you mean one of the following? [datatypeIds, targetId];
'Filter ((Contains('datatypeId, genetic_association) OR Contains('datatypeId, animal_model)) OR Contains('datatypeId, rna_expression))
+- Aggregate [targetId#1262], [targetId#1262, concat_ws(;, collect_list(datatypeId#1263, 0, 0)) AS datatypeIds#1278]
   +- Filter (diseaseId#1261 = EFO_0003767)
      +- Relation [diseaseId#1261,targetId#1262,datatypeId#1263,score#1264,evidenceCount#1265L] parquet


### Export dataset with known IBD drug targets

In [9]:
drug_targets = evidence_ibd.filter(col("datasourceId").contains("chembl"))
drug_targets.count()

275

In [10]:
drug_targets_pd = drug_targets.toPandas()
drug_targets_pd

,datatypeId,datasourceId,diseaseId,targetId,score,evidenceCount
0,known_drug,chembl,EFO_0003767,ENSG00000004779,0.165492,3
1,known_drug,chembl,EFO_0003767,ENSG00000006210,0.143539,3
2,known_drug,chembl,EFO_0003767,ENSG00000007314,0.788195,7
3,known_drug,chembl,EFO_0003767,ENSG00000010671,0.151983,2
4,known_drug,chembl,EFO_0003767,ENSG00000011677,0.870818,7
...,...,...,...,...,...,...
270,known_drug,chembl,EFO_0003767,ENSG00000267855,0.165492,3
271,known_drug,chembl,EFO_0003767,ENSG00000268089,0.607931,1
272,known_drug,chembl,EFO_0003767,ENSG00000273079,0.030397,1
273,known_drug,chembl,EFO_0003767,ENSG00000274286,0.121586,1


In [11]:
drug_targets_pd.to_csv("ibd_drug_targets.csv", index=False)

# Calculate metric